# **Project 2:**

In [14]:
import numpy as np
import pandas as pd

## initialization

In [21]:
#patient type from 'A' to 'F'
patient_type = ['A', 'B', 'C', 'D', 'E']
bed_capacity = np.array([55,40,30,20,20])
arrivals_pr_day = np.array([14.5,11.0,8.0,6.5,5.0])
mean_length_of_stay = np.array([2.9,4.0,4.5,1.4,3.9])
urgency_points = np.array([1,2,3,4,5])

#reloaction probability
relocation_probability = np.array([
    [0.0, 0.05, 0.10, 0.05, 0.80, 0.00],
    [0.2, 0, 0.50, 0.15, 0.15, 0.00],
    [0.30, 0.20, 0, 0.20, 0.30, 0.00],
    [0.35, 0.30, 0.05, 0, 0.3, 0.00],
    [0.20, 0.10, 0.60 ,0.10, 0, 0.00],
    [0.20, 0.20, 0.20, 0.20, 0.20 ,0]
    ])

#Dataframe containing all the data (indexed by patient type)
df = pd.DataFrame({
    'Patient Type': patient_type,
    'Bed Capacity': bed_capacity,
    'Arrivals per day': arrivals_pr_day,
    'Mean Length of Stay': mean_length_of_stay,
    'Urgency Points': urgency_points
    }).set_index('Patient Type')
df

,Bed Capacity,Arrivals per day,Mean Length of Stay,Urgency Points
Patient Type,,,,
A,55,14.5,2.9,1
B,40,11.0,4.0,2
C,30,8.0,4.5,3
D,20,6.5,1.4,4
E,20,5.0,3.9,5


In [34]:
class Ward:
    def __init__(self, type, capacity, urgency_points):
        """
        Args:
            type (Char): Ward type (A, B, C, D, E)
            capacity (int): Maximum number of patients that can be admitted to the ward
            urgency_points (int): Urgency points associated with the ward
        """
        self.type = type
        self.capacity = capacity
        self.current_occupancy = 0
        self.urgency_points = urgency_points
        
    @classmethod
    def from_dataframe(cls, df, type):
        """
        Initialization method for the Ward object that takes in a dataframe and a ward type and returns a Ward object with the corresponding data from the dataframe.
        
        Args:
            df (pandas.dataframe): _description_
            type (Char): Ward type (A, B, C, D, E)
        Returns:
            Ward: Ward object with the corresponding data from the dataframe
        """
        type = type
        capacity = df["Bed Capacity"][type]
        urgency_points = df["Urgency Points"][type]
        return cls(type, capacity, urgency_points)
    
    def admit(self):
        """ admits a patient to the ward if there is space

        Returns:
            Bool: Whether a patient was admitted or not
        """
        if self.current_occupancy < self.capacity:
            self.current_occupancy += 1
            return True
        else:
            return False
        
    def discharge(self):
        """discharges a patient from the ward if there is a patient to discharge

        Returns:
            Bool: Whether a patient was discharged or not
        """
        if self.current_occupancy > 0:
            self.current_occupancy -= 1
            return True
        else:
            return False
        
    def __repr__(self):
        return f"{self.type} Ward with {self.capacity} beds and {self.current_occupancy} patients currently admitted."
    def __str__(self):
        return f"{self.type} Ward with {self.capacity} beds and {self.current_occupancy} patients currently admitted."
    

def initialize_wards(df):
    """
    Initializes the wards using the data from the dataframe
    
    Args:
        df (pandas.dataframe): Dataframe containing the data for the wards
    Returns:
        list: List of Ward objects
    """
    wards = []
    for ward_type in df.index:
        ward = Ward.from_dataframe(df, ward_type)
        wards.append(ward)
    return wards

In [36]:
wards = initialize_wards(df) #list of wards
#print the wards
for ward in wards:
    print(ward)

A Ward with 55 beds and 0 patients currently admitted.
B Ward with 40 beds and 0 patients currently admitted.
C Ward with 30 beds and 0 patients currently admitted.
D Ward with 20 beds and 0 patients currently admitted.
E Ward with 20 beds and 0 patients currently admitted.


In [ ]:
class Patient:
    def __init__(self, arrival_time, occupancy_time):
        """
        Args:
            arrival_time (float): Time at which the patient arrives
            occupancy_time (float): Time at which the patient occupies a bed
        """
        self.occupancy_time = occupancy_time
        self.rejected = False
        
        #event
        self.next_event = "Admission"
        self.next_event_time = arrival_time